# Create and Train Simple Neural Network Model

## Data Preparation (Sequence Vectorization)

In [1]:
import time

import tensorflow as tf
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import json
import re

import nltk
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

### Import News Title and Content as Input and News Label as Output

In [2]:
start_time = time.time()

npz_title = np.load('Title_Data.npz',allow_pickle=True)
npz_content = np.load('Content_Data.npz',allow_pickle=True)

title = npz_title['inputs']
content = npz_content['inputs']
output = npz_title['targets']

title.shape[0], output.shape[0]

(9805, 9805)

### Text Preprocessing
1. Remove Special Characters using Regular Expressions
2. Tokenize text
3. Word Normalization (Lemmatization)
4. Remove Stopwords

In [3]:
stopword = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

cleaned_title = []
cleaned_content = []

for i in range(title.shape[0]):
    title_sentence = re.sub('[^A-Za-z0-9 ]+', '', title[i])
    title_tokens = nltk.word_tokenize(title_sentence)
    title_lemmatized_word = [lemmatizer.lemmatize(word) for word in title_tokens]
    title_removed_stopwords = [word for word in title_lemmatized_word if word not in stopword]
    cleaned_title.append(" ".join(word.lower() for word in title_removed_stopwords))

for j in range(content.shape[0]):
    content_sentence = re.sub('[^A-Za-z0-9 ]+', '', content[j])
    content_tokens = nltk.word_tokenize(content_sentence)
    content_lemmatized_word = [lemmatizer.lemmatize(word) for word in content_tokens]
    content_removed_stopwords = [word for word in content_lemmatized_word if word not in stopword]
    cleaned_content.append(" ".join(word.lower() for word in content_removed_stopwords))
    
cleaned_title = np.asarray(cleaned_title,dtype=object)
cleaned_content = np.asarray(cleaned_content,dtype=object)

len(cleaned_title), len(cleaned_content), len(output)

(9805, 9805, 9805)

### Split Train and Test Data

In [4]:
X_train_title, X_test_title, y_train_title, y_test_title = train_test_split(cleaned_title, output, test_size = 0.2, random_state = 1)
X_train_content, X_test_content, y_train_content, y_test_content = train_test_split(cleaned_content, output, test_size = 0.2, random_state = 1)

### Feed News Title and Content into Keras Tokenizer

In [5]:
max_vocab_title = 15000
max_vocab_content = 30000

tokenizer_title = Tokenizer(num_words = max_vocab_title)
tokenizer_title.fit_on_texts(X_train_title)

tokenizer_content = Tokenizer(num_words = max_vocab_content)
tokenizer_content.fit_on_texts(X_train_content)

### Save all Tokenized words into a (.json) Dictionary

In [6]:
dictionary_title = tokenizer_title.word_index
with open('dictionary_title.json', 'w') as dictionary_title_file:
    json.dump(dictionary_title, dictionary_title_file)
    
dictionary_content = tokenizer_content.word_index
with open('dictionary_content.json', 'w') as dictionary_content_file:
    json.dump(dictionary_content, dictionary_content_file)

### Convert Text to Word Sequence based on Dictionary (And Make All Texts The Same Length) 

In [7]:
def convert_text_to_index_array_title(text):
    return [dictionary_title[word] for word in kpt.text_to_word_sequence(text)]

def convert_text_to_index_array_content(text):
    return [dictionary_content[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices_title = []
allWordIndices_content = []

for text_title in X_train_title:
    wordIndices_title = convert_text_to_index_array_title(text_title)
    allWordIndices_title.append(wordIndices_title)

for text_content in X_train_content:
    wordIndices_content = convert_text_to_index_array_content(text_content)
    allWordIndices_content.append(wordIndices_content)

### Cast the Word Indices into Numpy Arrays (For Any Future Usage)

In [8]:
allWordIndices_title = np.asarray(allWordIndices_title)
allWordIndices_content = np.asarray(allWordIndices_content)

### Create One-Hot Matrices From the Word Indices and Make the Output Categorical

In [9]:
train_title_x = tokenizer_title.sequences_to_matrix(allWordIndices_title, mode='binary')
train_content_x = tokenizer_content.sequences_to_matrix(allWordIndices_content, mode='binary')

train_title_y = tf.keras.utils.to_categorical(y_train_title, 2)
train_content_y = tf.keras.utils.to_categorical(y_train_content, 2)

### Create and Train a Simple Neural Network Model with News Title

In [26]:
model_title = tf.keras.Sequential()

model_title.add(tf.keras.layers.Dense(64, input_shape=(max_vocab_title,), activation = 'relu'))
model_title.add(tf.keras.layers.Dropout(0.2))
model_title.add(tf.keras.layers.Dense(16,activation='relu'))
model_title.add(tf.keras.layers.Dropout(0.2))
model_title.add(tf.keras.layers.Dense(2,activation='relu'))
model_title.add(tf.keras.layers.Activation('sigmoid'))

model_title.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', 'Recall'])

batch_size = 500
max_epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model_title.fit(train_title_x, train_title_y, batch_size = batch_size, epochs = max_epochs, callbacks=[early_stopping], validation_split = 0.2 , shuffle=True, verbose=1)

Train on 6275 samples, validate on 1569 samples
Epoch 1/10
6275/6275 [==============================] - 1s 186us/sample - loss: 0.6913 - accuracy: 0.5461 - Recall: 0.5202 - val_loss: 0.6864 - val_accuracy: 0.5809 - val_Recall: 0.8674
Epoch 2/10
6275/6275 [==============================] - 1s 97us/sample - loss: 0.6744 - accuracy: 0.6316 - Recall: 0.9331 - val_loss: 0.6673 - val_accuracy: 0.6150 - val_Recall: 0.9681
Epoch 3/10
6275/6275 [==============================] - 1s 97us/sample - loss: 0.6391 - accuracy: 0.6993 - Recall: 0.9791 - val_loss: 0.6400 - val_accuracy: 0.6670 - val_Recall: 0.9598
Epoch 4/10
6275/6275 [==============================] - 1s 99us/sample - loss: 0.5905 - accuracy: 0.7615 - Recall: 0.9847 - val_loss: 0.6154 - val_accuracy: 0.6899 - val_Recall: 0.9465
Epoch 5/10
6275/6275 [==============================] - 1s 98us/sample - loss: 0.5414 - accuracy: 0.8163 - Recall: 0.9857 - val_loss: 0.5989 - val_accuracy: 0.7135 - val_Recall: 0.9395
Epoch 6/10
6275/6275 [====

### Create and Train a Simple Neural Network Model with News Content

In [11]:
model_content = tf.keras.Sequential()

model_content.add(tf.keras.layers.Dense(128, input_shape=(max_vocab_content,), activation = 'relu'))
model_content.add(tf.keras.layers.Dropout(0.2))
model_content.add(tf.keras.layers.Dense(16,activation='relu'))
model_content.add(tf.keras.layers.Dropout(0.2))
model_content.add(tf.keras.layers.Dense(2,activation='relu'))
model_content.add(tf.keras.layers.Activation('sigmoid'))

model_content.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', 'Recall'])

batch_size = 500
max_epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model_content.fit(train_content_x, train_content_y, batch_size = batch_size, epochs = max_epochs, callbacks=[early_stopping], validation_split = 0.2 , shuffle=True, verbose=1)

Train on 6275 samples, validate on 1569 samples
Epoch 1/10
6275/6275 [==============================] - 2s 327us/sample - loss: 0.6363 - accuracy: 0.6651 - Recall: 0.7500 - val_loss: 0.5608 - val_accuracy: 0.8152 - val_Recall: 0.9375
Epoch 2/10
6275/6275 [==============================] - 1s 217us/sample - loss: 0.5127 - accuracy: 0.8305 - Recall: 0.9578 - val_loss: 0.5046 - val_accuracy: 0.8706 - val_Recall: 0.9433
Epoch 3/10
6275/6275 [==============================] - 1s 211us/sample - loss: 0.4461 - accuracy: 0.9022 - Recall: 0.9790 - val_loss: 0.4798 - val_accuracy: 0.8932 - val_Recall: 0.9401
Epoch 4/10
6275/6275 [==============================] - 1s 210us/sample - loss: 0.4081 - accuracy: 0.9425 - Recall: 0.9879 - val_loss: 0.4851 - val_accuracy: 0.8917 - val_Recall: 0.9567
Epoch 5/10
6275/6275 [==============================] - 1s 210us/sample - loss: 0.3872 - accuracy: 0.9620 - Recall: 0.9931 - val_loss: 0.4841 - val_accuracy: 0.8932 - val_Recall: 0.9516


### Transform Test Data into Similar Format to Feed into Neural Network Model created for News Title

In [12]:
# create another Tokenizer for model testing 
tokenizer_test_title = Tokenizer(num_words = max_vocab_title)

# this utility makes sure that all the words in your input
# are registered in the dictionary
# before trying to turn them into a matrix.
def convert_text_to_index_array_title_test(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary_title:
            wordIndices.append(dictionary_title[word])
    return wordIndices

allWordIndices_test_title = []

for text_test in X_test_title:
    wordIndices_test = convert_text_to_index_array_title_test(text_test)
    allWordIndices_test_title.append(wordIndices_test)

# create one-hot matrices out of the indexed train_title_input
test_title_x = tokenizer_test_title.sequences_to_matrix(allWordIndices_test_title, mode='binary')

# treat the labels as categories
test_title_y = tf.keras.utils.to_categorical(y_test_title, 2)

### Convert Test Data into Similar Format to Feed into Neural Network Model Trained for News Content

In [13]:
# create another Tokenizer for model testing 
tokenizer_test_content = Tokenizer(num_words = max_vocab_content)

# this utility makes sure that all the words in your input
# are registered in the dictionary
# before trying to turn them into a matrix.
def convert_text_to_index_array_content_test(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary_content:
            wordIndices.append(dictionary_content[word])
    return wordIndices

allWordIndices_test_content = []

for text_test in X_test_content:
    wordIndices_test = convert_text_to_index_array_content_test(text_test)
    allWordIndices_test_content.append(wordIndices_test)

# create one-hot matrices out of the indexed train_content_input
test_content_x = tokenizer_test_content.sequences_to_matrix(allWordIndices_test_content, mode='binary')

# treat the labels as categories
test_content_y = tf.keras.utils.to_categorical(y_test_title, 2)

### Test Performance on Neural Network Model (Accuracy)

In [14]:
model_title_loss, model_title_accuracy, model_title_recall = model_title.evaluate(test_title_x, test_title_y)
model_content_loss, model_content_accuracy, model_content_recall = model_content.evaluate(test_content_x, test_content_y)

1961/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1961/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [15]:
model_title_loss, model_title_accuracy, model_title_recall

(0.6043482263254791, 0.7491076, 0.9153493)

In [16]:
model_content_loss, model_content_accuracy, model_content_recall

(0.4843757978846138, 0.8954615, 0.9551249)

In [17]:
y_pred_title = model_title.predict_classes(test_title_x)

con_mat_title = tf.math.confusion_matrix(labels=y_test_title, predictions=y_pred_title).numpy()

y_pred_content = model_content.predict_classes(test_content_x)

con_mat_content = tf.math.confusion_matrix(labels=y_test_content, predictions=y_pred_content).numpy()

In [18]:
con_mat_title

array([[793, 188],
       [258, 722]])

In [19]:
con_mat_content

array([[917,  64],
       [113, 867]])

In [20]:
end_time = time.time()

time_elapsed = end_time - start_time
time_elapsed

70.1442940235138

In [21]:


sequence_model_performance = {
                                "Title_Accuracy": str(model_title_accuracy),
                                "Title_Recall": str(model_title_recall),
                                "Content_Accuracy": str(model_content_accuracy),
                                "Content_Recall": str(model_content_recall),
                                "Time": str(time_elapsed)
                             }


with open('Model_Sequence.json', 'w') as Model_Sequence_File:
    json.dump(sequence_model_performance, Model_Sequence_File)